In [1]:
#Plotting
%matplotlib inline
from matplotlib import style
style.use("fivethirtyeight")
import matplotlib.pyplot as plt

In [2]:
#Data manipulation
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [3]:
#Python SQL Toolkit and ORM
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import session
from sqlalchemy import create_engine, func

In [4]:
#Create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [5]:
#New base class
Base = automap_base()

#Reflect the tables from the hawaii.sqlite database
Base.prepare(engine)

In [6]:
#View classes found by automap
Base.classes.keys()

['measurement', 'station']

In [8]:
#Class references
Measurement = Base.classes.measurement
Station = Base.classes.station